In [60]:
import pandas as pd

In [61]:
result = pd.read_csv('./total_padded_encoded_df.csv')
data = pd.DataFrame(result)
data.head()

,Unnamed: 0,genre,1,2,3,4,5,6,7,8,...,843,844,845,846,847,848,849,850,851,852
0,0,1,0,2,3,0,0,4,0,5,...,1,1,1,1,1,1,1,1,1,1
1,1,1,31,41,42,43,44,45,0,46,...,1,1,1,1,1,1,1,1,1,1
2,2,1,138,139,140,0,0,0,141,0,...,1,1,1,1,1,1,1,1,1,1
3,3,1,117,0,203,0,204,205,206,0,...,1,1,1,1,1,1,1,1,1,1
4,4,1,284,0,285,286,287,0,288,288,...,1,1,1,1,1,1,1,1,1,1


# feature, label 분리

In [62]:
feature_df = data.iloc[:, 2:]
label_df = data[['genre']]
print(feature_df.shape, label_df.shape)

(772, 852) (772, 1)


In [63]:
feature_df.head()

,1,2,3,4,5,6,7,8,9,10,...,843,844,845,846,847,848,849,850,851,852
0,0,2,3,0,0,4,0,5,6,7,...,1,1,1,1,1,1,1,1,1,1
1,31,41,42,43,44,45,0,46,0,47,...,1,1,1,1,1,1,1,1,1,1
2,138,139,140,0,0,0,141,0,143,0,...,1,1,1,1,1,1,1,1,1,1
3,117,0,203,0,204,205,206,0,118,207,...,1,1,1,1,1,1,1,1,1,1
4,284,0,285,286,287,0,288,288,289,290,...,1,1,1,1,1,1,1,1,1,1


In [64]:
label_df.head()

,genre
0,1
1,1
2,1
3,1
4,1


# DataSet 생성, Tensor 변환, DataLoader 생성

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_df, label_df, stratify=label_df, test_size=0.2, random_state=42)

In [66]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [67]:
print(f"Train {X_train.shape}     {y_train.shape}")
print(f"Test  {X_test.shape}      {y_test.shape}")

Train (617, 852)     (617, 1)
Test  (155, 852)      (155, 1)


In [68]:
num_train = int(round(int(len(data))*0.9, 0))
num_train

695

In [69]:
from torch.utils.data.dataset import random_split

In [70]:
import torch
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.x_data = X
        self.y_data = y
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self,idx):
        x = torch.LongTensor(self.x_data.iloc[idx])
        y = torch.LongTensor(self.y_data.iloc[idx])
        return x, y

In [71]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [72]:
from torch.utils.data import DataLoader

BATCH = 20
train_loader = DataLoader(train_dataset, batch_size = BATCH, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = BATCH, shuffle = True)

# 모델 설계

In [73]:
import torch.nn as nn
class music_rnn(nn.Module):
    def __init__(self, VOCAB_SIZE, EMBED_DIM, HIDDEN_SIZE, NUM_CLASS):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=EMBED_DIM)
        self.rnn = nn.RNN(input_size=EMBED_DIM, hidden_size=HIDDEN_SIZE, batch_first = True)
        # 학습 모델을 RNN 대신에 LSTM을 사용해볼수도 있겠다
        # RNN에서 양방향 학습 파라미터를 설정해볼 수 도 있겠다.
        
        # 활성화함수 => 다중 분류니까
        # self.fc = nn.Softmax(HIDDEN_SIZE, NUM_CLASS) # 이렇게 쓰는게 아닌가?
        self.fc = nn.Linear(HIDDEN_SIZE, NUM_CLASS) # 다중 분류인데 이게 되나?
        
        self.init_weights() # 이건 왜 쓴거지?
        self.dropout = nn.Dropout()  # 혹시 모를 과대적합을 위해서
        
    # 가중치 초기화
    def init_weights(self):
        range = 0.5
        self.embedding.weight.data.uniform_(-range, range)
        # self.fc.weight.data.uniform_(-range, range)
        # self.fc.bias.data.zero_()
        
    # 순방향 학습 진행
    def forward(self, text):
        embed = self.embedding(text)
        output, hidden = self.rnn(embed)
        result = self.fc(output)                # 이 층을 왜 통과시켜줘야하는거지? ;;
        # result = F.softmax(self.fc(output), dim=1)
        return result

In [74]:
# 학습용 변수들 지정
from torch import optim
import torch
import torch.nn as nn
import torch.nn.functional as F
VOCAB_SIZE = 13714
EMBED_DIM = 50
HIDDEN_SIZE = 10
NUM_CLASS = 8
model = music_rnn(VOCAB_SIZE, EMBED_DIM, HIDDEN_SIZE, NUM_CLASS)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(params = model.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# 함수 지정 (train, test)

In [76]:
import torchmetrics.functional.classification as metrics
import numpy as np
def training(DATALOADER):
    model.train()
    output_list = [[], [], [], [], []]
    # loss, acc, precision, recall, f1_score
    for data in DATALOADER:
        feature = data[0]
        label = data[1]
        label = F.one_hot(label)
        print(label)
        print('-'*50)

        train_feature = feature.to(device)
        train_label = label.to(device)
        
        pred = model(train_feature)

        # 손실 검정
        print(f"label.shape - {label.shape}   pred.shape - {pred.shape}")
        loss = loss_fn(pred, train_label) 
        output_list[0].append(loss.item())
        output_list[1].append(metrics.accuracy(pred, train_label, task = 'binary').item())
        output_list[2].append(metrics.precision(pred, train_label, task = 'binary').item())
        output_list[3].append(metrics.recall(pred, train_label, task = 'binary').item())
        output_list[4].append(metrics.f1_score(pred, train_label, task = 'binary').item())
        
        # 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return output_list
training(train_loader)

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 1]],

        [[0, 0, 0, 0, 0, 0, 0, 1, 0]],

        [[0, 1, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 1]],

        [[0, 1, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 1, 0, 0, 0]],

        [[0, 0, 0, 0, 1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 1, 0, 0]],

        [[0, 0, 1, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 1]],

        [[0, 0, 0, 0, 0, 0, 0, 1, 0]],

        [[0, 0, 0, 0, 0, 0, 1, 0, 0]],

        [[0, 0, 0, 0, 0, 1, 0, 0, 0]],

        [[0, 0, 1, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0, 0]]])
--------------------------------------------------
label.shape - torch.Size([20, 1, 9])   pred.shape - torch.Size([20, 852, 8])


RuntimeError: Expected target size [20, 8], got [20, 1, 9]